In [1]:
import pandas as pd
import numpy as np

In [2]:
cols = ['timestamp', 'hostname', 'record', 'TTL','DNS_IP','ANS_IP', '2LD']
df = pd.read_csv('data/raw_logs_auth_2016_sep_12_00_14', sep='\t', names= cols, nrows=5500000)
df = df.sort_values(['timestamp'])
df['tunnel'] = 0

In [3]:
# get the count of A records as those are good indicators of tunneling
t = df[df['record'] != 'A']
suspect_domains = t.groupby(['2LD']).size().sort_values(ascending=False)

In [4]:
domains = suspect_domains.to_frame().index.tolist()[:1000]

In [5]:
for domain in domains:
    print domain
    # limit the df to the suspected domain
    domain_df = df[(df['2LD'] == domain)]
    grp_df = domain_df.groupby(['record', 'timestamp']).size().to_frame()
    grp_df = grp_df.reset_index()
    
    # get the time difference
    grp_df['delta_size'] = np.abs(grp_df[0] - grp_df[0].shift(1))
    grp_df['delta_time'] = grp_df['timestamp'] - grp_df['timestamp'].shift(1) 
    rec_time = grp_df.groupby(['record']).median()
    rec_type = rec_time.index.values
    time = rec_time[['delta_time']].values.reshape(1,-1)[0].tolist()
    m_time = zip(rec_type, time)
    
    for r,dt in m_time:
        sus_df = grp_df[(grp_df['record'] == r) & (grp_df['delta_time'] > dt) 
                        & (grp_df['delta_size'] > 100)]
        if len(sus_df) > 0:
            ts = sus_df['timestamp'].values
            if len(ts) > 2:
                for i in range(0,len(ts)):
                    if i < len(ts)-1:
                        sus = domain_df[(domain_df['timestamp'] >= ts[i]) & 
                                  (domain_df['timestamp'] < ts[i+1]) & 
                                  (domain_df['record'] == r) ]
                        df.set_value(sus.index, 'tunnel',1)

googlevideo.com.
akadns.net.
gvt1.com.
akamaihd.net.
cedexis.net.
edgesuite.net.
nsatc.net.
trafficmanager.net.
xvideos.com.
rackcdn.com.
google.com.
sharepoint.com.
yahoo.com.
trouter.io.
edgecastcdn.net.
mail.ru.
force.com.
tc.qq.com.
rhcloud.com.
blob.core.windows.net.
azurewebsites.windows.net.
2mdn.net.
nest.com.
azurewebsites.net.
footprintdns.com.
linkedin.com.
lidetc.com.
sina.com.cn.
wscdns.com.
mozilla.org.
apple.com.
salesforce.com.
meraki.com.
360.cn.
g.aaplimg.com.
icloud.com.
xiaomi.com.
taobao.com.
inscname.net.
edgecastdns.net.
ctmail.com.
my.salesforce.com.
ebay.com.
youtube.com.
exelator.com.
ksmobile.net.
avito.st.
dropbox.com.
alibaba.com.
msn.com.
gmail.com.
jomodns.com.
miui.com.
network-auth.com.
baidu.com.
jd.com.
wshifen.com.
aol.com.
webex.com.
sinaedge.com.
outlook.com.
vidible.tv.
cdn20.com.
microsoft.com.
pusher.com.
mozilla.net.
picsart.com.
sinaimg.cn.
live.com.
lenovomm.com.
sogou.com.
adnxs.com.
files.wordpress.com.
windowsupdate.com.
iqiyi.com.
kxcdn.c

In [ ]:
df = df[~df['hostname'].str.contains("apple.com")]

In [ ]:
df = df[~df['hostname'].str.contains("google.com")]

In [ ]:
df = df[~df['hostname'].str.contains("yahoo.com")]

In [ ]:
df = df[~df['hostname'].str.contains("dropbox.com")]

In [ ]:
df = df[~df['hostname'].str.contains("twitter.com")]

In [ ]:
tunnels_df = tunnels_df[~tunnels_df['hostname'].str.contains("vimeo.com")]

In [11]:
#tunnels_df = df[df['tunnel'] == 1] akadns.net. #throw thsi index out 3453857
df[df ['tunnel'] == 0]

,timestamp,hostname,record,TTL,DNS_IP,ANS_IP,2LD,tunnel
1767796,1473646800,df66652dcbeae667.cname.365cyd.cn.,A,300.0,116.211.121.141,117.21.219.79,365cyd.cn.,0
1754568,1473646800,thyme.comcastnets.net.,CNAME,300.0,thyme.mts.nbcuni.com.,209.133.100.150,comcastnets.net.,0
1754569,1473646800,www.nasawestprime.com.,CNAME,30.0,iznasa.hs.llnwd.net.,205.251.193.107,nasawestprime.com.,0
1754570,1473646800,redbud3.sopinet.com.,A,300.0,51.254.148.43,2400:cb00:2049:1::adf5:3b7d,sopinet.com.,0
1754571,1473646800,www.apk256.com.,A,100.0,195.22.26.238,54.77.72.254,apk256.com.,0
1754572,1473646800,dreamserver.no-ip.org.,A,60.0,84.83.85.13,50.31.129.129,no-ip.org.,0
1754573,1473646800,zenit-bet.org.,A,600.0,160.153.129.34,2607:f208:302::31,zenit-bet.org.,0
1754574,1473646800,telegramboobsbot-bfbdev.rhcloud.com.,CNAME,60.0,ex-std-node536.prod.rhcloud.com.,208.78.70.23,rhcloud.com.,0
1754567,1473646800,prod-cache-web-lb-1757567688.eu-west-1.elb.ama...,A,60.0,54.154.5.9,205.251.194.227,eu-west-1.elb.amazonaws.com.,0
1754575,1473646800,ex-std-node536.prod.rhcloud.com.,CNAME,150.0,ec2-54-173-168-158.compute-1.amazonaws.com.,208.78.70.23,rhcloud.com.,0


## Summary
Detecting DNS tunneling in this exercise relied mostly on statistical measures. One of the most noted indication of tunneling is observing the type of DNS record-- A, CNAME, MX, or TXT.  Other papers have shown that it is possible to tunnel A and CNAME records, traditionally, TXT records have been a more positive indicator. 

We start with counting the type of records, where at the 2LD domains with a high count of CNAME or MX records we will examine closer. With each suspicious domain, we get the frequency of requests per timestamp. We expect a high volume of requests during tunneling, and with a high latency between timestamps. Where we see a drop in requests and high latency, we mark that as tunneling. We also observe that the authorative IP response is quite consistent whcih we expect for a tunnel would attempt to go to the same authorative DNS. Another way to check is to observe the domain names in particular those with non-English words and many numerical characters.